In [1]:
import pandas as pd
import re
import regex  # 이모지 제거용

# ============================================
# 1) CSV 불러오기 (파일 경로만 바꿔서 사용)
# ============================================
FILE_PATH = "./reddit_추가/2차_현정3_No번역셀렉.csv"   
OUT_PATH  = "./reddit_추가/2차_현정3_No번역셀렉_cleand.csv"

# 인코딩 문제 방지용
try:
    df = pd.read_csv(FILE_PATH, encoding="utf-8-sig")
except UnicodeDecodeError:
    df = pd.read_csv(FILE_PATH, encoding="cp949")

print("원본 행 개수:", len(df))
print("컬럼 목록:", df.columns.tolist())
display(df.head())

원본 행 개수: 792
컬럼 목록: ['keyword', 'title', 'title_한글', 'content', 'content_한글', 'date', 'url']


,keyword,title,title_한글,content,content_한글,date,url
0,hard of hearing,88 Secrets Beneath the House,88 집 아래의 비밀,88 Secrets Beneath the House First we went ove...,NaN,2025-11-17 22:20:56,https://www.reddit.com/r/PotionsWillSaveMe/com...
1,hard of hearing,58 Travel Permit Part 3,58 여행 허가 파트 3,58 Travel Permit Part 3 Thanks to Kasumior may...,NaN,2025-11-17 22:05:01,https://www.reddit.com/r/PotionsWillSaveMe/com...
2,hardofhearing,Hearing parent with HOH child,HOH 자녀를 둔 청각 부모,Hello all I am a mom of a newly diagnosed chil...,NaN,2025-11-13 18:41:44,https://www.reddit.com/r/hardofhearing/comment...
3,hardofhearing,People look at me differently when I tell them...,내가 청각 장애가 있다고 말하면 사람들은 나를 다르게 봅니다.,Hi all Im 34 male I take care of myself Im ind...,NaN,2025-11-13 05:50:47,https://www.reddit.com/r/hardofhearing/comment...
4,hardofhearing,Pros and cons of rechargeable hearing aids,충전식 보청기의 장점과 단점,Looking to get new hearing aids in the next ye...,NaN,2025-10-31 18:47:26,https://www.reddit.com/r/hardofhearing/comment...


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 761 entries, 0 to 760
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   keyword     761 non-null    object 
 1   title       761 non-null    object 
 2   title_한글    760 non-null    object 
 3   content     761 non-null    object 
 4   content_한글  0 non-null      float64
 5   date        761 non-null    object 
 6   url         761 non-null    object 
dtypes: float64(1), object(6)
memory usage: 41.7+ KB


In [4]:
pip install googletrans==4.0.0-rc1 tqdm

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.5 MB/s eta 0:00:00
  DEPRECATION: Building 'googletrans' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'googletrans'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17454 sha256=fded9ce31c450c5de5db49fc38ed30231b55815ed08f06cd1b656f56d6672a74
  Stored in directory: /Users/hyeonjungnoh/Library/Caches/pip/wheels/c0/59/9f/7372f0cf70160fe61b528532e1a7c8498c4becd6bcffb022de
Successfully built googletrans
  Attempting uninstall: h11
    Found existing installation: h11 0.16.0
    Uninstalling h11-0

In [2]:
import pandas as pd
from deep_translator import GoogleTranslator
from tqdm import tqdm
import time

# ===========================================================
# 1. 파일 불러오기
# ===========================================================
FILE_PATH = "./reddit_추가/2차_현정3_No번역셀렉.csv"   
OUT_PATH  = "./reddit_추가/2차_현정3_번역완료_cleand.csv"
try:
    df = pd.read_csv(FILE_PATH, encoding="utf-8-sig")
except UnicodeDecodeError:
    df = pd.read_csv(FILE_PATH, encoding="cp949")

print(f"📂 파일 로드 완료: {len(df)}개 행")

# ===========================================================
# 2. 번역 함수 정의 (긴 글 분할 처리 추가)
# ===========================================================
def translate_content(text):
    # 빈 값 체크
    if pd.isna(text) or str(text).strip() == "":
        return ""
    
    text_str = str(text)
    
    # 💡 핵심 수정: 4500자 이상이면 쪼개서 번역 (안전하게 4500자로 설정)
    if len(text_str) > 4500:
        chunks = [text_str[i:i+4500] for i in range(0, len(text_str), 4500)]
        translated_chunks = []
        
        for chunk in chunks:
            try:
                # 잘린 조각 번역
                res = GoogleTranslator(source='auto', target='ko').translate(chunk)
                translated_chunks.append(res)
                time.sleep(0.5) # 긴 글 처리 중이니 조금 더 쉼
            except Exception as e:
                print(f"⚠️ 긴 글 부분 번역 실패: {e}")
                translated_chunks.append(chunk) # 실패하면 원본 그대로 붙임
        
        return " ".join(translated_chunks) # 조각들을 다시 하나로 합침

    else:
        # 4500자 이하면 한 번에 번역
        try:
            return GoogleTranslator(source='auto', target='ko').translate(text_str)
        except Exception as e:
            print(f"⚠️ 번역 실패: {e}")
            return text_str

# ===========================================================
# 3. 번역 실행
# ===========================================================
print("🚀 번역 시작 (긴 글 자동 분할 기능 적용)...")
tqdm.pandas()

results = []
# tqdm으로 진행 상황 보기
for text in tqdm(df['content']):
    res = translate_content(text)
    results.append(res)
    time.sleep(0.2) # 기본 딜레이

# 번역 결과를 컬럼에 저장
df['content_한글'] = results

# ===========================================================
# 4. 컬럼 정리 및 저장
# ===========================================================
print("\n🧹 컬럼 정리 중...")

target_cols = ['keyword', 'title_한글', 'content_한글', 'date', 'url']
available_cols = [c for c in target_cols if c in df.columns]
final_df = df[available_cols].copy()

# 이름 변경 (한글 꼬리표 떼기)
final_df = final_df.rename(columns={
    'title_한글': 'title',
    'content_한글': 'content'
})

print("\n[최종 결과 미리보기]")
print(final_df.head())

final_df.to_csv(OUT_PATH, index=False, encoding="utf-8-sig")
print(f"\n✅ 저장 완료: {OUT_PATH}")

📂 파일 로드 완료: 792개 행
🚀 번역 시작 (긴 글 자동 분할 기능 적용)...


100%|███████████████████████████████████████| 792/792 [1:28:46<00:00,  6.73s/it]


🧹 컬럼 정리 중...

[최종 결과 미리보기]
           keyword                               title  \
0  hard of hearing                         88 집 아래의 비밀   
1  hard of hearing                       58 여행 허가 파트 3   
2    hardofhearing                     HOH 자녀를 둔 청각 부모   
3    hardofhearing  내가 청각 장애가 있다고 말하면 사람들은 나를 다르게 봅니다.   
4    hardofhearing                     충전식 보청기의 장점과 단점   

                                             content                 date  \
0  88 집 아래의 비밀 먼저 우리는 Ritachista가 기대했던 것과 이에 대해 우...  2025-11-17 22:20:56   
1  58 여행 허가서 3부 카스미오르 덕분에 좋은 채집 조건이 있었는지 아오기리 풀 찾...  2025-11-17 22:05:01   
2  안녕하세요 여러분 저는 새로 진단받은 중등도 청력 손실이 있는 2층 아이의 엄마입니...  2025-11-13 18:41:44   
3  안녕하세요 저는 34세 남성입니다. 저는 독립적입니다. 처음부터 인생을 구축했습니다...  2025-11-13 05:50:47   
4  내년에 새 보청기를 구입하려고 하는데 청능사로부터 유일한 옵션은 충전식이라고 들었습...  2025-10-31 18:47:26   

                                                 url  
0  https://www.reddit.com/r/PotionsWillSaveMe/com...  
1  https://www.reddit.com/r/PotionsWillSaveMe/com.

In [ ]:
# pip install --upgrade httpx httpcore

In [ ]:
# pip install deep-translator tqdm

In [3]:
final_df

,keyword,title,content,date,url
0,hard of hearing,88 집 아래의 비밀,88 집 아래의 비밀 먼저 우리는 Ritachista가 기대했던 것과 이에 대해 우...,2025-11-17 22:20:56,https://www.reddit.com/r/PotionsWillSaveMe/com...
1,hard of hearing,58 여행 허가 파트 3,58 여행 허가서 3부 카스미오르 덕분에 좋은 채집 조건이 있었는지 아오기리 풀 찾...,2025-11-17 22:05:01,https://www.reddit.com/r/PotionsWillSaveMe/com...
2,hardofhearing,HOH 자녀를 둔 청각 부모,안녕하세요 여러분 저는 새로 진단받은 중등도 청력 손실이 있는 2층 아이의 엄마입니...,2025-11-13 18:41:44,https://www.reddit.com/r/hardofhearing/comment...
3,hardofhearing,내가 청각 장애가 있다고 말하면 사람들은 나를 다르게 봅니다.,안녕하세요 저는 34세 남성입니다. 저는 독립적입니다. 처음부터 인생을 구축했습니다...,2025-11-13 05:50:47,https://www.reddit.com/r/hardofhearing/comment...
4,hardofhearing,충전식 보청기의 장점과 단점,내년에 새 보청기를 구입하려고 하는데 청능사로부터 유일한 옵션은 충전식이라고 들었습...,2025-10-31 18:47:26,https://www.reddit.com/r/hardofhearing/comment...
...,...,...,...,...,...
787,HearingLoss,군단 3권 27화 울란 전술,제27장 울란 전술 지구 시간 2206년 6월 13일 황궁의 알현실 루하즘 알현실에...,2025-11-13 19:53:50,https://www.reddit.com/r/u_Feeling_Pea5770/com...
788,HearingLoss,군단 3권 27화 울란 전술,제27장 울란 전술 지구 시간 2206년 6월 13일 황궁의 알현실 루하즘 알현실에...,2025-11-13 19:53:05,https://www.reddit.com/r/HFY/comments/1owb3bq/...
789,HearingLoss,스코치 지령 AU 복수의 균형 III pt1,2주도 안 돼서 Balance of Vengeance의 새로운 장이 열렸습니다htt...,2025-11-12 22:41:55,https://www.reddit.com/r/NatureofPredators/com...
790,HearingLoss,이비인후과 의사가 되는 방법,httpspreviewredditfuvmiatx7s0g1jpgwidth1024amp...,2025-11-12 07:56:30,https://www.reddit.com/r/u_Efficient-Tea-5841/...
